In [8]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [127]:
health_expdf =  pd.read_csv('Health Expenditure by Country.csv')  
health_expdf.head()
countryDF = health_expdf[['Country']].copy()
countryDF['Country'] = countryDF['Country'].str.upper()
countryDF.index = np.arange(1, len(countryDF)+1)
countryDF.index.name = 'Country_ID'
countryDF.head()

,Country
Country_ID,
1,AUSTRALIA
2,AUSTRIA
3,BELGIUM
4,CANADA
5,CHILE


In [112]:
health_expdf.index = np.arange(1, len(health_expdf)+1)
health_expdf = health_expdf[['health_expenditure']]
health_expdf.index.name = 'Country_ID'
health_expdf.head()

,health_expenditure
Country_ID,
1,9.334
2,10.408
3,10.349
4,10.790
5,9.102


In [124]:
def get_countryID(countryname):
    countryID = 0    
    selectedCountry = countryDF.loc[countryDF.Country == countryname.upper()]
    if(len(selectedCountry) > 0):
        countryID = selectedCountry.index.values.astype(int)[0]
    else:
        bestmatchscore = 0
        matchscore = 0
        bestmatchcountry = ''
        for index, row in countryDF.iterrows():
            matchscore = fuzz.ratio(countryname.upper(), row['Country'])

            if (matchscore > bestmatchscore):
                bestmatchscore = matchscore
                bestmatchcountry = row['Country']
        if (bestmatchscore> 80):
            selectedCountry = countryDF.loc[countryDF.Country == bestmatchcountry.upper()]
            countryID = selectedCountry.index.values.astype(int)[0]
        else:
            name_parts = countryname.split(' ') 
            if(len(name_parts)>1):
                for part in name_parts:
                    countryID = get_countryID(part)
                    if (countryID > 0):
                        break
            elif(len(countryname) < 4):
                # check for scenario US = United States                                
                for index, row in countryDF.iterrows():
                    shortname = ''
                    name_parts = row['Country'].split(' ')                    
                    if(len(name_parts) > 1):
                        #take the first letter of the word
                        for part in name_parts:
                            shortname = shortname + part[0]                        
                        if(shortname == countryname.upper()):
                            selectedCountry = countryDF.loc[countryDF.Country == row['Country']]
                            countryID = selectedCountry.index.values.astype(int)[0]
                            break                            

    return countryID

In [131]:
Covid_deathdf =  pd.read_csv('Covid deaths by country.csv')
Covid_deathdf.rename(columns={"Covid Deaths till 12th August, 2020": "Covid_Deaths"}, inplace = True)
Covid_deathdf['Country_ID'] = Covid_deathdf.apply (lambda row: get_countryID(row['Country']), axis=1)
Covid_deathdf = Covid_deathdf.loc[Covid_deathdf.Country_ID >0]
Covid_deathdf.set_index(['Country_ID'], inplace = True)
Covid_deathdf = Covid_deathdf[['Covid_Deaths']]
Covid_deathdf

,Covid_Deaths
Country_ID,
1,361
2,724
3,9900
4,9052
5,10205
6,13837
8,621
9,63
10,333
